In [1]:
from args import args
hparams = args()

In [2]:
from chatbot import Chatbot
chatbot = Chatbot(hparams)

Some weights of ChatbotModel were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.encoder.embed_tokens.weight', 'encoder.encoder.embed_positions.weight', 'encoder.encoder.layers.0.self_attn.k_proj.weight', 'encoder.encoder.layers.0.self_attn.k_proj.bias', 'encoder.encoder.layers.0.self_attn.v_proj.weight', 'encoder.encoder.layers.0.self_attn.v_proj.bias', 'encoder.encoder.layers.0.self_attn.q_proj.weight', 'encoder.encoder.layers.0.self_attn.q_proj.bias', 'encoder.encoder.layers.0.self_attn.out_proj.weight', 'encoder.encoder.layers.0.self_attn.out_proj.bias', 'encoder.encoder.layers.0.self_attn_layer_norm.weight', 'encoder.encoder.layers.0.self_attn_layer_norm.bias', 'encoder.encoder.layers.0.fc1.weight', 'encoder.encoder.layers.0.fc1.bias', 'encoder.encoder.layers.0.fc2.weight', 'encoder.encoder.layers.0.fc2.bias', 'encoder.encoder.layers.0.final_layer_norm.weight', 'encoder.encoder.layers.0.final_layer_norm.bias', 'encoder.encoder.

In [3]:
from chatbot.data import DataModule
dm = DataModule(hparams)
dm.prepare_data()
dm.setup('fit')

  0%|          | 0/17878 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

/home/insublee/anaconda3/envs/chatbot/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  0%|          | 0/105 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [4]:
# data check
batch = next(iter(dm.train_dataloader()))
for i in batch.keys():
    print(i, batch[i].size())

attention_mask torch.Size([16, 64])
decoder_attention_mask torch.Size([16, 64])
decoder_input_ids torch.Size([16, 64])
input_ids torch.Size([16, 64])
labels torch.Size([16, 64])
next_state_attention_mask torch.Size([16, 64])
next_state_input_ids torch.Size([16, 64])


In [5]:
# forward test
forward_out = chatbot(**batch)
print(forward_out.keys()) # odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])


In [6]:
# training step test
loss = chatbot.training_step(batch,0)
print(loss) # tensor(14.2616, grad_fn=<AddBackward0>)

/home/insublee/anaconda3/envs/chatbot/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


tensor(33.7003, grad_fn=<AddBackward0>)


In [7]:
# validation step test
loss = chatbot.validation_step(batch,0)
print(loss) # tensor(14.2616, grad_fn=<AddBackward0>)

{'loss': tensor(14.2776, grad_fn=<NllLossBackward>), 'bleu': 0.19241010443021755}


In [8]:
# generation test1. no input
#outputs = chatbot.model.generate(max_length=40)
#print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))
generated = chatbot.response()
print("Generated:", generated)

Generated: A few weeks ago, I wrote an article about how I would like to see my grandchildren grow up. Today, I am pleased to report that my grandchildren are growing up. I am


In [9]:
# generation test2. yes input
input_text = "hello my name is insub, and"
generated = chatbot.response(input_text)
print("Generated:", generated)

Generated: This is the first time in a long time that I have ever seen anything resembling an actual person. The second time was when I was a teenager. This time around, however, I


In [10]:
import pytorch_lightning as pl
# trainer loading and fit
#trainer = pl.Trainer(gpus=-1, auto_select_gpus=True)
#trainer = pl.Trainer(gpus=4, accelerator='ddp2')
#trainer = pl.Trainer(gpus=1, max_epochs=2)
trainer = pl.Trainer(gpus=1, max_epochs=1)
trainer.fit(chatbot, dm)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type         | Params
---------------------------------------
0 | model | ChatbotModel | 153 M 
---------------------------------------
152 M     Trainable params
1.5 M     Non-trainable params
153 M     Total params
615.741   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/home/insublee/anaconda3/envs/chatbot/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [11]:
# generation test3. after training no input
outputs = chatbot.model.generate(max_length=40)
print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated: In addition, there was also a segment of the public interest community known as Skyline, which is known known as the " "A").As far as technically speaking is concerned, we


In [12]:
# generation test4. after training yes input
input_text = "hi , how are you doing ? i'm getting ready to do some cheetah chasing to stay in shape ."
tokenized_text = dm.tokenizer([input_text], return_tensors="pt").input_ids
outputs = chatbot.model.generate(input_ids=tokenized_text, max_length=40)
print("Generated:", dm.tokenizer.decode(outputs[0], skip_special_tokens=True))

Generated: In neutral neutral neutral neutral to neutral to good solid solid solid base base in Western Europe, and neutral neutral in central Europe and neutral in to neutral in- neutral to the neutral neutral
